# WebAPI

ネットワークを経由してデータをやり取りする方法

In [1]:
import requests
import json
from pprint import pprint
import pandas as pd

In [2]:
def request(request_url, request_parameter:dict=None):
    res = requests.get(request_url, params=request_parameter)
    # 正常終了
    if res.status_code // 100 == 2:
        return res.json()
        
    # ネットワーク400番エラー(リクエストに失敗)
    elif res.status_code // 100 == 4:
        return res.json()
        
    # ネットワーク500番エラー(サーバ側に問題あり)
    elif res.status_code // 100 == 5:
        return res.json()

## 郵便番号検索APIを用いた例

|パラメータ名|項目名|必須|備考|
|:--|:--|:--:|:--|
|zipcode|郵便番号|〇|7桁の数字．ハイフン付きでも可．完全一致検索．|
|callback|コールバック関数名|-|JSONPとして出力する際のコールバック関数名．UTF-8でURLエンコードした文字列．|
|limit|最大件数|-|同一の郵便番号で複数県のデータが存在する場合に返される件数の上限値（数字）．|

In [3]:
post_address_prefix = 'https://zipcloud.ibsnet.co.jp/api/search'

In [4]:
res = request(post_address_prefix, {'zipcode' : '1000001'})
pprint(res)

{'message': None,
 'results': [{'address1': '東京都',
              'address2': '千代田区',
              'address3': '千代田',
              'kana1': 'ﾄｳｷｮｳﾄ',
              'kana2': 'ﾁﾖﾀﾞｸ',
              'kana3': 'ﾁﾖﾀﾞ',
              'prefcode': '13',
              'zipcode': '1000001'}],
 'status': 200}


## PokeAPIを用いた例
ポケモンの情報を取得できるAPI．

In [5]:
pokeAPI_prefix = 'https://pokeapi.co/api/v2/'

In [6]:
def extract_japanese(list_obj : list):
    keys = [dict_obj for dict_obj in list_obj if dict_obj['language']['name']=='ja']
    if len(keys) == 0:
        return None
    return keys

In [7]:
def extract_sword_shield(list_obj : list):
    keys = [dict_obj for dict_obj in list_obj if dict_obj['version_group']['name']=='sword-shield']
    if len(keys) == 0:
        return None
    return keys

In [8]:
type_df = pd.DataFrame(columns=['id', 'english_name', 'japanese_name'])

res_json = request(f'{pokeAPI_prefix}/type')

for type in res_json['results']:
    r_json = request(type['url'])
    add_df = pd.DataFrame(
        {
            'id'            : r_json['id'],
            'english_name'  : r_json['name'],
            'japanese_name' : None if extract_japanese(r_json['names']) is None else extract_japanese(r_json['names'])[0]['name'],
        },index=[r_json['id']]
    )
    type_df = pd.concat([type_df, add_df], axis=0)
    
type_df.set_index('id', inplace=True)
type_df.to_csv('../dataset/pokemon_type.csv')
display(type_df)

,english_name,japanese_name
id,,
1,normal,ノーマル
2,fighting,かくとう
3,flying,ひこう
4,poison,どく
5,ground,じめん
6,rock,いわ
7,bug,むし
8,ghost,ゴースト
9,steel,はがね


In [9]:
ability_df = pd.DataFrame(columns = ['id', 'japanese_name', 'flavor_text'])

res_json = request(f'{pokeAPI_prefix}/ability')

while res_json['next'] is not None:
    for ability in res_json['results']:
        r_json = request(ability['url'])

        add_df = pd.DataFrame(
            {
                'id'            : r_json['id'],
                'japanese_name' : None if extract_japanese(r_json['names']) is None else extract_japanese(r_json['names'])[0]['name'],
                'flavor_text'   : None if extract_japanese(r_json['flavor_text_entries']) is None else extract_sword_shield(extract_japanese(r_json['flavor_text_entries']))[0]['flavor_text']
            },index=[r_json['id']]
        )
        ability_df = pd.concat([ability_df, add_df], axis=0)
    res_json = request(res_json['next'])
    
ability_df.set_index('id', inplace=True)
# ability_df.dropna(how='any', inplace=True)
ability_df.to_csv('../dataset/pokemon_ability.csv')
display(ability_df)

,japanese_name,flavor_text
id,,
1,あくしゅう,臭い においを 放つことによって\n攻撃した ときに 相手を\nひるませることが ある。
2,あめふらし,登場 したときに\n天気を 雨に する。
3,かそく,毎ターン 素早さが 上がる。
4,カブトアーマー,硬い 甲羅に 守られて\n相手の 攻撃が\n急所に 当たらない。
5,がんじょう,相手の 技を 受けても\n一撃で 倒されることが ない。\n一撃必殺技も 効かない。
...,...,...
10049,None,None
10050,None,None
10051,None,None


In [10]:
def hidden_ability(list_obj):
    key = [dict_obj['ability']['url'].split('/')[-2] for dict_obj in list_obj if dict_obj['is_hidden']]
    if len(key) == 0:
        return None
    else:
        return key[0]

In [11]:
pokemon_df = pd.DataFrame(columns = ['id', 'japanese_name', 'english_name', 'genus', 'type1', 'type2', 'hp', 'Attack', 'Defence', 'Special-Attack', 'Special-Defence', 'Speed', 'ability1', 'ability2', 'hidden-ability'])

res_json = request(f'{pokeAPI_prefix}/pokemon')
    
while res_json['next'] is not None:
    for pokemon in res_json['results']:
        r = requests.get(pokemon['url'])
        p = requests.get(r.json()['species']['url'])
        add_df = pd.DataFrame(
            {
                'id' : r.json()['id'],
                'japanese_name' : [extract_japanese(p.json()['names'])[0]['name']],
                'english_name'  : r.json()['name'],
                'genus' : [extract_japanese(p.json()['genera'])[0]['genus']],
                'type1' : r.json()['types'][0]['type']['url'].split('/')[-2],
                'type2' : r.json()['types'][1]['type']['url'].split('/')[-2] if len(r.json()['types']) == 2 else None,
                'height' : r.json()['height'] / 10,
                'weight' : r.json()['weight'] / 10,
                'hp' : r.json()['stats'][0]['base_stat'],
                'Attack' : r.json()['stats'][1]['base_stat'],
                'Defence' : r.json()['stats'][2]['base_stat'],
                'Special-Attack' : r.json()['stats'][3]['base_stat'],
                'Special-Defence' : r.json()['stats'][4]['base_stat'],
                'Speed' : r.json()['stats'][5]['base_stat'],
                'ability1' : r.json()['abilities'][0]['ability']['url'].split('/')[-2],
                'ability2' : None if len(r.json()['abilities']) < 2 or r.json()['abilities'][1]['slot'] != 2 else r.json()['abilities'][1]['ability']['url'].split('/')[-2],
                'hidden-ability' : hidden_ability(r.json()['abilities'])
            }
        )
        pokemon_df = pd.concat([pokemon_df, add_df], axis=0)
    
    res_json = request(res_json['next'])

pokemon_df.set_index('id', inplace=True)
# pokemon_df.dropna(how='any', inplace=True)
pokemon_df.to_csv('../dataset/pokemon.csv')
display(pokemon_df)

,japanese_name,english_name,genus,type1,type2,hp,Attack,Defence,Special-Attack,Special-Defence,Speed,ability1,ability2,hidden-ability,height,weight
id,,,,,,,,,,,,,,,,
1,フシギダネ,bulbasaur,たねポケモン,12,4,45,49,49,65,65,45,65,None,34,0.7,6.9
2,フシギソウ,ivysaur,たねポケモン,12,4,60,62,63,80,80,60,65,None,34,1.0,13.0
3,フシギバナ,venusaur,たねポケモン,12,4,80,82,83,100,100,80,65,None,34,2.0,100.0
4,ヒトカゲ,charmander,とかげポケモン,10,None,39,52,43,60,50,65,66,None,94,0.6,8.5
5,リザード,charmeleon,かえんポケモン,10,None,58,64,58,80,65,80,66,None,94,1.1,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10231,ビリリダマ,voltorb-hisui,ボールポケモン,13,12,40,30,50,55,55,100,43,9,106,0.5,13.0
10232,マルマイン,electrode-hisui,ボールポケモン,13,12,60,50,70,80,80,150,43,9,106,1.2,71.0
10233,バクフーン,typhlosion-hisui,かざんポケモン,10,8,73,84,78,119,85,95,66,None,18,1.6,69.8
